In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report
import cv2


In [2]:
from google.colab import files

uploaded = files.upload()


Saving chickens.zip to chickens.zip
Saving ducks.zip to ducks.zip


In [3]:
import zipfile
import io

with zipfile.ZipFile(io.BytesIO(uploaded['chickens.zip']), 'r') as zip_ref:
    zip_ref.extractall()

with zipfile.ZipFile(io.BytesIO(uploaded['ducks.zip']), 'r') as zip_ref:
    zip_ref.extractall()


In [4]:
import os
import random
import shutil

# Function to split images and copy them to train, validation, and test directories
def split_images(original_dir, train_ratio, val_ratio, train_dir, val_dir, test_dir):
    # Create train, validation, and test directories if they don't exist
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Get the list of image filenames
    filenames = os.listdir(original_dir)
    # Shuffle the filenames randomly
    random.shuffle(filenames)
    # Calculate the number of images for the train set
    num_train = int(len(filenames) * train_ratio)
    # Calculate the number of images for the validation set
    num_val = int(len(filenames) * val_ratio)
    # Iterate over the filenames and copy the images to train, validation, or test directories
    for i, filename in enumerate(filenames):
        # Determine the destination directory
        if i < num_train:
            dest_dir = train_dir
        elif i < num_train + num_val:
            dest_dir = val_dir
        else:
            dest_dir = test_dir
        # Copy the image to the destination directory
        shutil.copy(os.path.join(original_dir, filename), os.path.join(dest_dir, filename))

# Set the paths to the original duck and chicken image directories
duck_dir = '/content/ducks/'
chicken_dir = '/content/chickens/'

# Set the paths to the new train, validation, and test directories
train_dir = '/content/train/'
val_dir = '/content/val/'
test_dir = '/content/test/'

# Split duck images
split_images(duck_dir, train_ratio=0.6, val_ratio=0.2, train_dir=train_dir+'ducks', val_dir=val_dir+'ducks', test_dir=test_dir+'ducks')

# Split chicken images
split_images(chicken_dir, train_ratio=0.6, val_ratio=0.2, train_dir=train_dir+'chickens', val_dir=val_dir+'chickens', test_dir=test_dir+'chickens')

print("Images split and copied successfully!")


Images split and copied successfully!


In [5]:
x_train=[]
train_path='/content/train'
test_path='/content/test'
val_path='/content/val'
for folder in os.listdir(train_path):

    sub_path=train_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(224,224))

        x_train.append(img_arr)

x_test=[]

for folder in os.listdir(test_path):

    sub_path=test_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(224,224))

        x_test.append(img_arr)

x_val=[]

for folder in os.listdir(val_path):

    sub_path=val_path+"/"+folder

    for img in os.listdir(sub_path):

        image_path=sub_path+"/"+img

        img_arr=cv2.imread(image_path)

        img_arr=cv2.resize(img_arr,(224,224))

        x_val.append(img_arr)

In [6]:
train_x=np.array(x_train)
test_x=np.array(x_test)
val_x=np.array(x_val)
train_x=train_x/255.0
test_x=test_x/255.0
val_x=val_x/255.0

In [7]:
# Set up data generators
train_datagen = ImageDataGenerator(rescale=1./255)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Generate batches of training, validation, and test data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 120 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [8]:
# Load pre-trained VGG16 model without the top classification layer
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = False

80134624/80134624 [==============================] - 1s 0us/step


In [10]:
# Add custom classification head on top of the base model
x = Flatten()(base_model.output)
# x = Dense(256, activation='relu')(x)
# x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)  # Make sure the output matches the number of classes

# Create final model
model = Model(base_model.input, output)


In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [12]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)

In [13]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    callbacks=[early_stop],
    validation_data=val_generator,
    validation_steps=len(val_generator)
)


Epoch 1/10
4/4 [==============================] - 120s 33s/step - loss: 0.8775 - accuracy: 0.5250 - val_loss: 0.5901 - val_accuracy: 0.6750
Epoch 2/10
4/4 [==============================] - 128s 33s/step - loss: 0.6149 - accuracy: 0.6167 - val_loss: 0.5629 - val_accuracy: 0.7000
Epoch 3/10
4/4 [==============================] - 119s 32s/step - loss: 0.4772 - accuracy: 0.7500 - val_loss: 0.5420 - val_accuracy: 0.8000
Epoch 4/10
4/4 [==============================] - 103s 29s/step - loss: 0.4169 - accuracy: 0.7917 - val_loss: 0.6287 - val_accuracy: 0.6750
Epoch 5/10
4/4 [==============================] - 118s 34s/step - loss: 0.3152 - accuracy: 0.8667 - val_loss: 0.5664 - val_accuracy: 0.7250
Epoch 6/10
4/4 [==============================] - 121s 35s/step - loss: 0.2586 - accuracy: 0.8917 - val_loss: 0.5842 - val_accuracy: 0.7000
Epoch 7/10
4/4 [==============================] - 118s 32s/step - loss: 0.2419 - accuracy: 0.9250 - val_loss: 0.5380 - val_accuracy: 0.7250
Epoch 8/10
4/4 [====

In [24]:
# Predict on the validation set
val_images, val_labels = next(val_generator)
predictions = model.predict(val_images)
predicted_classes = [1 if prob > 0.5 else 0 for prob in predictions]

# Generate classification report
print(classification_report(val_labels, predicted_classes, target_names=['Chicken', 'Duck']))


1/1 [==============================] - 27s 27s/step
              precision    recall  f1-score   support

     Chicken       0.91      0.56      0.69        18
        Duck       0.62      0.93      0.74        14

    accuracy                           0.72        32
   macro avg       0.76      0.74      0.72        32
weighted avg       0.78      0.72      0.71        32

